In [ ]:
!pip install boto3

In [1]:
import os
import sys
import boto3
import logging
import shutil
import threading
from botocore.client import Config
from matplotlib import pyplot as plt
from botocore.exceptions import ClientError
from boto3.s3.transfer import TransferConfig

In [2]:
END_POINT_URL = 'http://uvo17qqh4jn92xmchpj.vm.cld.sr'
A_KEY = 'AKIAtEpiGWUcQIelPRlD1Pi6xQ'
S_KEY = 'YNV6xS8lXnCTGSy1x2vGkmGnmdJbZSapNXaSaRhK'

TODO: Move to separate class

In [3]:
class ProgressPercentage(object):
    def __init__(self, filename):
        self._filename = filename
        self._size = float(os.path.getsize(filename))
        self._seen_so_far = 0
        self._lock = threading.Lock()
        
    def __call__(self, bytes_amount):
        # To simplify, assume this is hooked up to a single filename
        with self._lock:
            self._seen_so_far += bytes_amount
            percentage = (self._seen_so_far / self._size) * 100
            sys.stdout.write("\r%s  %s / %s  (%.2f%%)" %
                             (self._filename, self._seen_so_far, 
                              self._size, percentage))
            sys.stdout.flush()

def multipart_upload_with_s3(bucket_name, file_path=None, object_name=None):
    # Multipart upload (see notes)
    config = TransferConfig(multipart_threshold=1024 * 25, max_concurrency=10,
                            multipart_chunksize=1024 * 25, use_threads=True)
    key_path = 'multipart_files/{}'.format(object_name)
    print(bucket_name,file_path,object_name,key_path)
    s3_client.upload_file(file_path, bucket_name, key_path,
                          ExtraArgs={'ACL': 'public-read', 
                                     'ContentType': 'text/pdf'},
                          Config=config, Callback=ProgressPercentage(file_path))
    
def multipart_download_with_s3(bucket_name, file_path=None, object_name=None):
    config = TransferConfig(multipart_threshold=1024 * 25, max_concurrency=10,
                            multipart_chunksize=1024 * 25, use_threads=True)
    temp_file = os.path.dirname(__file__)
    s3_resource.Object(bucket_name, 
                       object_name
                       ).download_file(file_path, Config=config,
                                       Callback=ProgressPercentage(temp_file))

"""Functions for buckets operation"""
def create_bucket_op(bucket_name, region):
    if region is None:
        s3_client.create_bucket(Bucket=bucket_name)
    else:
        location = {'LocationConstraint': region}
        s3_client.create_bucket(Bucket=bucket_name, 
                                CreateBucketConfiguration=location)

def list_bucket_op(bucket_name, region, operation):
    buckets = s3_client.list_buckets()
    if buckets['Buckets']:
        for bucket in buckets['Buckets']:
            print(bucket)
            return True
    else:
        logging.error('unknown bucket operation')
        return False
    
def bucket_operation(bucket_name, region=None, operation='list'):
    try:
        if operation == 'delete':
            s3_client.delete_bucket(Bucket=bucket_name)
        elif operation == 'create':
            create_bucket_op(bucket_name, region)
        elif operation == 'list':
            return list_bucket_op(bucket_name, region, operation)
        else:
            logging.error('unknown bucket operation')
            return False
    except ClientError as e:
        logging.error(e)
        return False
    return True

"""Functions for objects operation"""
def list_object_op(bucket_name):
     s3_objects = s3_client.list_objects_v2(Bucket=bucket_name)
     if s3_objects.get('Contents'):
         for obj in s3_objects['Contents']:
             print(obj)

def delete_object_op(bucket_name, object_name, operation):
    if not object_name:
        logging.error('object_name missing for {}'.format(operation))
        return False  
    s3_client.delete_object(Bucket=bucket_name, Key=object_name)
    return True

def upload_download_object_op(bucket_name, object_name, file_path, operation):
    if not file_path or not object_name:
        logging.error('file_path and/or object_name missing for upload')
        return False
    if operation == 'upload':
        multipart_upload_with_s3(bucket_name=bucket_name, file_path=file_path,
                                 object_name=object_name)
    else:
        multipart_download_with_s3(bucket_name=bucket_name, file_path=file_path,
                                   object_name=object_name)
    return True

def object_operation(bucket_name=None, object_name=None, file_path=None,
                     operation='list'):                                                             
    try:
        if not bucket_name:
            logging.error('The bucket name %s is missing for %s operation!'
                          % (bucket_name, operation))
            return False
        if operation == 'list':
            list_object_op(bucket_name)
        elif operation == 'delete':
            return delete_object_op(bucket_name, object_name, operation)
        elif operation == 'upload' or operation == 'download':
            return upload_download_object_op(bucket_name, object_name,
                                             file_path, operation)      
        else:
            logging.error('unknown object operation')
            return False
    except ClientError as e:
        logging.error(e)
        return False
    return True

"""Functions for files operation"""
def list_op_file(bucket_name):
    current_bucket = s3_resource.Bucket(bucket_name)
    print('The files in bucket %s:\n' % (bucket_name))
    for obj in current_bucket.objects.all():
        print(obj.meta.data) 
        
    return True

def delete_op_file(bucket_name, file_name, operation):
    if not file_name:
        logging.error('The file name %s is missing for%s operation!' 
                      % (file_name, operation))
        return False
    s3_client.delete_object(Bucket=bucket_name, Key=file_name)
    return True
    
def upload_download_op_file(bucket_name, file_name, file_location,
                            region, operation):
    if not file_location:
        logging.error('The file location %d is missing for %s operation!'
                      % (file_location, operation))
        return False
    if operation == 'download':
        s3_resource.Bucket(bucket_name).download_file(file_name, file_location)
    elif operation == 'upload' and region is None:
        s3_resource.Bucket(bucket_name).upload_file(file_location, file_name)
    else:
         location = {'LocationConstraint': region}
         s3_resource.Bucket(bucket_name
                            ).upload_file(file_location, file_name,
                                          CreateBucketConfiguration=location) 
    return True
    
def file_operation(bucket_name=None, file_name=None, file_location=None, 
                   region=None, operation='list'):
    if not bucket_name:
        logging.error('The bucket name is %s missing!' % (bucket_name))
        return False 
    try:
        if operation == 'list':
            return list_op_file(bucket_name)
        elif operation == 'delete':
            return delete_op_file(bucket_name, file_name, operation)  
        elif operation == 'upload' or operation == 'download':
            return upload_download_op_file(bucket_name, file_name, 
                                           file_location, region, operation)
        else:
            logging.error('unknown file operation')
            return False  
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [4]:
s3_resource = boto3.resource('s3', endpoint_url=END_POINT_URL,
                             aws_access_key_id=A_KEY,
                             aws_secret_access_key=S_KEY,
                             config=Config(signature_version='s3v4'),
                             region_name='US')

s3_client = boto3.client('s3', endpoint_url=END_POINT_URL,
                         aws_access_key_id=A_KEY,
                         aws_secret_access_key=S_KEY,
                         config=Config(signature_version='s3v4'),
                         region_name='US')

In [5]:
files = list_op_file('testbucket')

The files in bucket testbucket:

{'Key': 'PennFudanPed.zip', 'LastModified': datetime.datetime(2021, 4, 27, 13, 17, 28, tzinfo=tzutc()), 'ETag': '"f0c664d97f7b031b9394255fa323543f-7"', 'Size': 53723336, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'S3user', 'ID': 'e500ea6b45f64f068ab001b7f1fdfc57ed2faed247474d81b66f69a6233727c8'}}
{'Key': 'multipart_files/randomfile.txt', 'LastModified': datetime.datetime(2021, 4, 27, 7, 59, 15, tzinfo=tzutc()), 'ETag': '"9d37ffb157b1d1a31ec0220f1095a926"', 'Size': 13, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'S3user', 'ID': 'e500ea6b45f64f068ab001b7f1fdfc57ed2faed247474d81b66f69a6233727c8'}}
{'Key': 'randomfile.txt', 'LastModified': datetime.datetime(2021, 4, 27, 7, 50, 55, tzinfo=tzutc()), 'ETag': '"9d37ffb157b1d1a31ec0220f1095a926"', 'Size': 13, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'S3user', 'ID': 'e500ea6b45f64f068ab001b7f1fdfc57ed2faed247474d81b66f69a6233727c8'}}


In [ ]:
bucket_name = 'testbucket'
file_name = 'randomfile.txt'
path_file_upload = '/home/dreamchild/randomfile.txt'
path_file_download = '/tmp/randomfile.txt'

In [ ]:
print('random file {} and {}'.format(1,2))

In [ ]:
if file_operation(bucket_name, file_name, path_file_upload, None, 'upload'):
    print('file {} uploaded successfully'.format(file_name))

Upload dataset to Cortx

## Setup filename and data paths

In [6]:
bucket_name = 'testbucket'
file_name = 'PennFudanPed.zip'
path_file_upload = '/home/dreamchild/sgcortex/data/{}'.format(file_name)
path_file_download = '/home/dreamchild/sgcortex/tmp/{}'.format(file_name)

Upload to Cortx

In [14]:
if file_operation(bucket_name, file_name, path_file_upload, None, 'upload'):
    print('file {} uploaded to Cortx'.format(file_name))

file PennFudanPed.zip uploaded to Cortx


### List files in bucket

In [7]:
list_op_file('testbucket')

The files in bucket testbucket:

{'Key': 'PennFudanPed.zip', 'LastModified': datetime.datetime(2021, 4, 27, 13, 17, 28, tzinfo=tzutc()), 'ETag': '"f0c664d97f7b031b9394255fa323543f-7"', 'Size': 53723336, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'S3user', 'ID': 'e500ea6b45f64f068ab001b7f1fdfc57ed2faed247474d81b66f69a6233727c8'}}
{'Key': 'multipart_files/randomfile.txt', 'LastModified': datetime.datetime(2021, 4, 27, 7, 59, 15, tzinfo=tzutc()), 'ETag': '"9d37ffb157b1d1a31ec0220f1095a926"', 'Size': 13, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'S3user', 'ID': 'e500ea6b45f64f068ab001b7f1fdfc57ed2faed247474d81b66f69a6233727c8'}}
{'Key': 'randomfile.txt', 'LastModified': datetime.datetime(2021, 4, 27, 7, 50, 55, tzinfo=tzutc()), 'ETag': '"9d37ffb157b1d1a31ec0220f1095a926"', 'Size': 13, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'S3user', 'ID': 'e500ea6b45f64f068ab001b7f1fdfc57ed2faed247474d81b66f69a6233727c8'}}


True

In [ ]:
status = object_operation(bucket_name, object_name=file_name, file_path=path_file_upload, operation='upload')

download from cortex

In [22]:
if file_operation(bucket_name, file_name, path_file_download, None, operation='download'):
    print('file {} downloaded successfully.'.format(file_name))

file PennFudanPed.zip downloaded successfully.


## unzip dataset

In [23]:
from zipfile import ZipFile

In [24]:
with ZipFile(path_file_download, 'r') as dataset_zip_file:
    dataset_zip_file.extractall('/home/dreamchild/sgcortex/data/unzipped')

## create dataset

In [8]:
import os
import numpy as np
import torch
from PIL import Image


class PennFudanDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)
        mask = np.array(mask)
        obj_ids = np.unique(mask)
        obj_ids = obj_ids[1:]

        masks = mask == obj_ids[:, None, None]

        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

### setup pretrained model

In [9]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

num_classes = 2
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /home/dreamchild/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


### change model backbone

In [10]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

backbone = torchvision.models.mobilenet_v2(pretrained=True).features
backbone.out_channels = 1280

anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)


model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /home/dreamchild/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


### setup instance segmentation model

In [32]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor


def get_model_instance_segmentation(num_classes):
    
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

### data augmentation helper functions

In [22]:
from torchvision import transforms as T
from torch.utils.data import DataLoader

In [20]:
def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

### testing the forward method

In [34]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
dataset = PennFudanDataset('/home/dreamchild/sgcortex/data/unzipped/PennFudanPed', get_transform(train=True))
data_loader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4)
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)   # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/dreamchild/anaconda3/envs/fastai/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/dreamchild/anaconda3/envs/fastai/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/dreamchild/anaconda3/envs/fastai/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-8-7c503c66062b>", line 53, in __getitem__
    img, target = self.transforms(img, target)
TypeError: __call__() takes 2 positional arguments but 3 were given


In [31]:
from torchvision.engine import train_one_epoch, evaluate
import torch.utils as utils

def main():
    # train on the GPU or on the CPU, if a GPU is not available
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # our dataset has two classes only - background and person
    num_classes = 2
    # use our dataset and defined transformations
    dataset = PennFudanDataset('PennFudanPed', get_transform(train=True))
    dataset_test = PennFudanDataset('PennFudanPed', get_transform(train=False))

    # split the dataset in train and test set
    indices = torch.randperm(len(dataset)).tolist()
    dataset = torch.utils.data.Subset(dataset, indices[:-50])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

    # define training and validation data loaders
    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=2, shuffle=True, num_workers=4,
        collate_fn=utils.collate_fn)

    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)

    # get the model using our helper function
    model = get_model_instance_segmentation(num_classes)

    # move model to the right device
    model.to(device)

    # construct an optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005,
                                momentum=0.9, weight_decay=0.0005)
    # and a learning rate scheduler
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                   step_size=3,
                                                   gamma=0.1)

    # let's train it for 10 epochs
    num_epochs = 10

    for epoch in range(num_epochs):
        # train for one epoch, printing every 10 iterations
        train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
        # update the learning rate
        lr_scheduler.step()
        # evaluate on the test dataset
        evaluate(model, data_loader_test, device=device)

    print("That's it!")

ModuleNotFoundError: No module named 'torchvision.engine'

In [26]:
print(torch.__version__)

1.7.1
